# import libraries

In [57]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from collections import defaultdict
import time

# read and clean data

In [58]:
df = pd.read_csv('diabetic_data.csv')
df.shape

(101766, 50)

In [59]:
df.isna().sum()

encounter_id                    0
patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

In [60]:
(df.shape[0] - df[['max_glu_serum']].isna().sum()) / df.shape[0]

max_glu_serum    0.052532
dtype: float64

In [61]:
(df.shape[0] - df[['A1Cresult']].isna().sum()) / df.shape[0]

A1Cresult    0.167227
dtype: float64

In [62]:
# remove max_glu_serum and A1Cresult
df.dropna(axis=1, inplace=True)
df.shape

(101766, 48)

In [63]:
df.isna().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
weight                      0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
payer_code                  0
medical_specialty           0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose  

In [64]:
# check columns got "?" values
df.astype(str).eq("?").sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide                       0
glyburide     

In [65]:
# remove weight. it has big number of "?" values
df.drop(columns=['weight'], inplace=True)

In [66]:
df.shape

(101766, 47)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 47 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   admission_type_id         101766 non-null  int64 
 6   discharge_disposition_id  101766 non-null  int64 
 7   admission_source_id       101766 non-null  int64 
 8   time_in_hospital          101766 non-null  int64 
 9   payer_code                101766 non-null  object
 10  medical_specialty         101766 non-null  object
 11  num_lab_procedures        101766 non-null  int64 
 12  num_procedures            101766 non-null  int64 
 13  num_medications           101766 non-null  int64 
 14  numb

In [68]:
for column in ['race', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'age']:
    print("*" * 10)
    print(f"column {column} unique values: {df[column].unique()}")

**********
column race unique values: ['Caucasian' 'AfricanAmerican' '?' 'Other' 'Asian' 'Hispanic']
**********
column payer_code unique values: ['?' 'MC' 'MD' 'HM' 'UN' 'BC' 'SP' 'CP' 'SI' 'DM' 'CM' 'CH' 'PO' 'WC' 'OT'
 'OG' 'MP' 'FR']
**********
column medical_specialty unique values: ['Pediatrics-Endocrinology' '?' 'InternalMedicine'
 'Family/GeneralPractice' 'Cardiology' 'Surgery-General' 'Orthopedics'
 'Gastroenterology' 'Surgery-Cardiovascular/Thoracic' 'Nephrology'
 'Orthopedics-Reconstructive' 'Psychiatry' 'Emergency/Trauma'
 'Pulmonology' 'Surgery-Neuro' 'Obsterics&Gynecology-GynecologicOnco'
 'ObstetricsandGynecology' 'Pediatrics' 'Hematology/Oncology'
 'Otolaryngology' 'Surgery-Colon&Rectal' 'Pediatrics-CriticalCare'
 'Endocrinology' 'Urology' 'Psychiatry-Child/Adolescent'
 'Pediatrics-Pulmonology' 'Neurology' 'Anesthesiology-Pediatric'
 'Radiology' 'Pediatrics-Hematology-Oncology' 'Psychology' 'Podiatry'
 'Gynecology' 'Oncology' 'Pediatrics-Neurology' 'Surgery-Plastic'
 'Su

In [69]:
df.readmitted.unique()

array(['NO', '>30', '<30'], dtype=object)

In [70]:
# remove rows containing values starting with letters
# for column in ['diag_1', 'diag_2', 'diag_3']:
#     df = df[~df[column].str.startswith(('E', 'V'))]
# df.shape

In [71]:
bin_midpoints = {
    '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35,
    '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75,
    '[80-90)': 85, '[90-100)': 95
}

df['age_numeric'] = df['age'].map(bin_midpoints)
df.drop(columns=['age'], inplace=True)

In [72]:
df['readmitted_binary'] = df['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

y = df['readmitted_binary']
print(df['readmitted_binary'].value_counts())
X = df.drop(columns=['readmitted', 'readmitted_binary'])
X.shape, y.shape

readmitted_binary
0    90409
1    11357
Name: count, dtype: int64


((101766, 46), (101766,))

In [73]:
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features

['race',
 'gender',
 'payer_code',
 'medical_specialty',
 'diag_1',
 'diag_2',
 'diag_3',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed']

In [74]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.to_list()
numeric_features

['encounter_id',
 'patient_nbr',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses',
 'age_numeric']

In [75]:
# the pipline uses imputer to replace ? with mose frequent values
preprocessor = ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), numeric_features), 
        ("cat", Pipeline([
            ("imputer", SimpleImputer(missing_values="?", strategy="most_frequent")),
            ("encoder", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features),
    ]
)

# Fit and transform the DataFrame
X_processed = preprocessor.fit_transform(X)
X_processed.shape

(101766, 2439)

In [76]:
# For numeric features, their names remain the same
num_features_names = numeric_features

# For categorical features, get the names from the OneHotEncoder
cat_transformer = preprocessor.named_transformers_['cat']
ohe = cat_transformer.named_steps['encoder']

# This handles underscores in original feature names correctly
cat_features_names = ohe.get_feature_names_out(categorical_features)

# Combine numeric + one-hot encoded categorical features
all_feature_names = np.concatenate([num_features_names, cat_features_names])
all_feature_names.shape

(2439,)

# Train models

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

In [78]:
models = {}

for leaves in [100, 300]:
    models[f"rf_{leaves}"] = RandomForestClassifier(n_estimators=100, max_leaf_nodes=leaves, random_state=42)
    models[f"xgb_{leaves}"] = XGBClassifier(
                                    n_estimators=100,
                                    max_leaves=leaves,
                                    learning_rate=0.1,
                                    random_state=42
                                )

In [79]:
print("training...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"training {name}\n")
    model.fit(X_train, y_train)
    print(f"it took {time.time() - start_time} seconds to train model {name}")
    print("*" * 10)
    print()


training...

training rf_100

it took 24.04747986793518 seconds to train model rf_100
**********

training xgb_100

it took 0.9255125522613525 seconds to train model xgb_100
**********

training rf_300

it took 39.89363431930542 seconds to train model rf_300
**********

training xgb_300

it took 0.8288600444793701 seconds to train model xgb_300
**********



In [80]:
print("predicting...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_pred = model.predict(X_test)
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


predicting...

predicting rf_100

Sensitivity of rf_100: 0.00000000000000000000
it took 0.6445870399475098 seconds to predicting model rf_100
**********

predicting xgb_100

Sensitivity of xgb_100: 0.02014598540145985314
it took 0.08082914352416992 seconds to predicting model xgb_100
**********

predicting rf_300

Sensitivity of rf_300: 0.00000000000000000000
it took 0.8374135494232178 seconds to predicting model rf_300
**********

predicting xgb_300

Sensitivity of xgb_300: 0.02014598540145985314
it took 0.04614973068237305 seconds to predicting model xgb_300
**********



In [81]:
values, count = np.unique(y_test, return_counts=True)
print("y_test: ", values, count)
values, count = np.unique(y_pred, return_counts=True)
print("y_pred: ",values, count)

y_test:  [0 1] [27105  3425]
y_pred:  [0 1] [30420   110]


In [26]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.2
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


use lover threshold instead of default 0.5 threshold.
predicting...

predicting rf_100

Sensitivity of rf_100: 0.01897810218978102093
it took 0.5414936542510986 seconds to predicting model rf_100
**********

predicting xgb_100

Sensitivity of xgb_100: 0.21956204379562044293
it took 0.04430246353149414 seconds to predicting model xgb_100
**********

predicting rf_300

Sensitivity of rf_300: 0.04554744525547445300
it took 0.7415182590484619 seconds to predicting model rf_300
**********

predicting xgb_300

Sensitivity of xgb_300: 0.21956204379562044293
it took 0.04388308525085449 seconds to predicting model xgb_300
**********



# train again with  class weights

In [27]:
models = {}

# scale_pos_weight = (#negative samples) / (#positive samples)
scale = (y_train == 0).sum() / (y_train == 1).sum()

for leaves in [100, 300]:
    models[f"rf_{leaves}"] = RandomForestClassifier(n_estimators=100, max_leaf_nodes=leaves, class_weight='balanced', random_state=42)
    models[f"xgb_{leaves}"] = XGBClassifier(
                                    n_estimators=100,
                                    max_leaves=leaves,
                                    learning_rate=0.1,
                                    scale_pos_weight=scale,
                                    random_state=42
                                )

In [ ]:
print("training...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"training {name}\n")
    model.fit(X_train, y_train)
    print(f"it took {time.time() - start_time} seconds to train model {name}")
    print("*" * 10)
    print()


training...

training rf_100

it took 24.718871355056763 seconds to train model rf_100
**********

training xgb_100

it took 0.7993874549865723 seconds to train model xgb_100
**********

training rf_300



In [ ]:
print("predicting...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_pred = model.predict(X_test)
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.3
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    training_data=X_train,
    feature_names=all_feature_names,
    class_names=['No disease','Disease'],
    mode='classification'
)

exp = explainer.explain_instance(X_test[0], models['rf_300'].predict_proba)
exp.show_in_notebook(show_table=True)


In [ ]:
from sklearn.inspection import PartialDependenceDisplay

pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("classifier", models['rf_300'])
])

PartialDependenceDisplay.from_estimator(
    pipeline,          # preprocessor + model
    X[:100],         # original DataFrame
    features=[
        "time_in_hospital",
        "num_medications",
        "num_lab_procedures"
    ]
)

In [ ]:
X[:100].shape

# Smoting

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
smote_models = {}

for leaves in [100, 300]:
    smote_models[f"rf_{leaves}"] = RandomForestClassifier(n_estimators=100, max_leaf_nodes=leaves, random_state=42)
    smote_models[f"xgb_{leaves}"] = XGBClassifier(
                                    n_estimators=100,
                                    max_leaves=leaves,
                                    learning_rate=0.1,
                                    random_state=42
                                )

In [ ]:
print("training...\n")
for name, model in smote_models.items():
    start_time = time.time()
    print(f"training {name}\n")
    model.fit(X_train, y_train)
    print(f"it took {time.time() - start_time} seconds to train model {name}")
    print("*" * 10)
    print()


In [ ]:
print("predicting...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_pred = model.predict(X_test)
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.1
for name, model in smote_models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


# Select important features

In [ ]:
importances = models['rf_100'].feature_importances_


In [ ]:
feature_importances = pd.DataFrame({
    'feature': all_feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

feature_importances.head(20)

In [ ]:
important_features = feature_importances.feature[:20].tolist()
important_features_copy = []
for name in important_features:
    if name not in X.columns.tolist():
        print(name)
    else:
        important_features_copy.append(name)

for name in ['diag_3', 'diag_2', 'insulin', 'diabetesMed']:
    important_features_copy.append(name)

len(important_features_copy)

In [ ]:
numeric_features_copy = []
categorical_features_copy = []

for name in important_features_copy:
    if name in num_features_names:
        numeric_features_copy.append(name)
    elif name in categorical_features:
        categorical_features_copy.append(name)


len(numeric_features_copy), len(categorical_features_copy)

In [ ]:
numeric_features_copy

In [ ]:
categorical_features_copy

In [ ]:

# the pipline uses imputer to replace ? with mose frequent values
preprocessor = ColumnTransformer([
        ("num", SimpleImputer(strategy="median"), numeric_features_copy), 
        ("cat", Pipeline([
            ("imputer", SimpleImputer(missing_values="?", strategy="most_frequent")),
            ("encoder", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_features_copy),
    ]
)

# Fit and transform the DataFrame
X_processed = preprocessor.fit_transform(X)
X_processed.shape

# train it again with selected features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)

In [ ]:
models = {}

# scale_pos_weight = (#negative samples) / (#positive samples)
scale = (y_train == 0).sum() / (y_train == 1).sum()

for leaves in [100, 300]:
    models[f"rf_{leaves}"] = RandomForestClassifier(n_estimators=100, max_leaf_nodes=leaves, class_weight='balanced', random_state=42)
    models[f"xgb_{leaves}"] = XGBClassifier(
                                    n_estimators=100,
                                    max_leaves=leaves,
                                    learning_rate=0.1,
                                    scale_pos_weight=scale,
                                    random_state=42
                                )

In [ ]:
print("training...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"training {name}\n")
    model.fit(X_train, y_train)
    print(f"it took {time.time() - start_time} seconds to train model {name}")
    print("*" * 10)
    print()



In [ ]:
print("predicting...\n")
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_pred = model.predict(X_test)
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()


In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.3
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    sensitivity = recall_score(y_test, y_pred, pos_label=1)  # 1 = positive class
    print(f"Sensitivity of {name}: {sensitivity:.20f}")
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()

In [ ]:
from joblib import dump, load

# Save the trained model (and optionally preprocessor)
for name, model in models.items():
    saved_model_name = f"{name}.joblib"
    dump(model, saved_model_name)

dump(preprocessor, 'preprocessor.joblib')
dump(num_features_names, "num_features_names.joblib")
dump(cat_features_names, "cat_features_names.joblib")

In [ ]:
# Later, load it back
# clf_loaded = load('random_forest_model.joblib')
# preprocessor_loaded = load('preprocessor.joblib')

In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.3
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_train)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    print(f'classification_report: {classification_report(y_train, y_pred)}')
    auc = roc_auc_score(y_train, y_proba)
    print("ROC-AUC:", auc)
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()

In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.3
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    print(f'classification_report: {classification_report(y_test, y_pred)}')
    auc = roc_auc_score(y_test, y_proba)
    print("ROC-AUC:", auc)
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()

In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.5
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_train)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    print(f'classification_report: {classification_report(y_train, y_pred)}')
    auc = roc_auc_score(y_train, y_proba)
    print("ROC-AUC:", auc)
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()

In [ ]:
print("use lover threshold instead of default 0.5 threshold.")
print("predicting...\n")
threshold = 0.5
for name, model in models.items():
    start_time = time.time()
    print(f"predicting {name}\n")
    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_proba >= threshold).astype(int)  # lower threshold
    print(f'classification_report: {classification_report(y_test, y_pred)}')
    auc = roc_auc_score(y_test, y_proba)
    print("ROC-AUC:", auc)
    print(f"it took {time.time() - start_time} seconds to predicting model {name}")
    print("*" * 10)
    print()

# by analyzing the metrics we see the model could not be trained completely UNDERFIT!!